# Bert Pipeline

In [1]:
!pip uninstall -y kfp
! pip install kfp captum torchvision matplotlib pillow pytorch-lightning flask flask-compress ipywidgets minio

Found existing installation: kfp 1.0.4
Uninstalling kfp-1.0.4:
  Successfully uninstalled kfp-1.0.4
     |████████████████████████████████| 222 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 21.5 MB/s eta 0:00:01
     |████████████████████████████████| 806 kB 95.7 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 83.7 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 96.6 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 93.3 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 93.8 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 91.4 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 2.5 MB/s  eta 0:00:01
     |████████████████████████████████| 52 kB 60.3 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 51.1 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |██████████████

In [2]:
# Node Version
! node --version

# Install yarn
! npm install -g npm
! npm install --global yarn
! yarn --version

v14.16.0
/usr/bin/npm -> /usr/lib/node_modules/npm/bin/npm-cli.jssill gently link link does not alreaK
/usr/bin/npx -> /usr/lib/node_modules/npm/bin/npx-cli.js
+ npm@7.14.0m......] - postinstall:just-diff: info lifecycle just-diff@3.1.1~po
added 60 packages from 23 contributors, removed 241 packages and updated 194 packages in 4.744s
              ] / reify:yarn: sill audit bulk request { yarn: [ '1.22.10'0'
added 1 package, and audited 2 packages in 746ms

found 0 vulnerabilities
1.22.10


In [3]:
# Install Jupyter Notebook Widgets
! jupyter nbextension install --py --symlink --sys-prefix captum.insights.attr_vis.widget
! jupyter nbbextension install jupyter-matplotlib

Installing /opt/conda/lib/python3.8/site-packages/captum/insights/attr_vis/widget/static -> jupyter-captum-insights
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable captum.insights.attr_vis.widget --py --sys-prefix
    
Traceback (most recent call last):
  File "/opt/conda/bin/jupyter", line 8, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.8/site-packages/jupyter_core/command.py", line 285, in main
    command = _jupyter_abspath(subcommand)
  File "/opt/conda/lib/python3.8/site-packages/jupyter_core/command.py", line 124, in _jupyter_abspath
    raise Exception(
Exception: Jupyter command `jupyter-nbbextension` not found.


In [4]:
# Enable Jupyter Notebook Extensions
! jupyter nbextension enable --py widgetsnbextension
! jupyter nbextension enable captum.insights.attr_vis.widget --py --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension jupyter-captum-insights/extension...
      - Validating: OK


In [5]:
import kfp
import json
import os
from kfp.onprem import use_k8s_secret
from kfp import components
from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler

import numpy as np
import logging

from PIL import Image
import torchvision.transforms as transforms

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F

import captum
from captum.attr import IntegratedGradients, Occlusion, LayerGradCam, LayerAttribution
from captum.attr import visualization as viz

kfp.__version__

'1.6.2'

# Enter your gateway and the cookie
[Use this extension on chrome to get token]( https://chrome.google.com/webstore/detail/editthiscookie/fngmhnnpilhplaeedifhccceomclgfbg?hl=en)

![image.png](./image.png)

## Update values for the ingress gateway and auth session

In [6]:
INGRESS_GATEWAY='http://istio-ingressgateway.istio-system.svc.cluster.local'
AUTH="MTYyMjAzMTgwNHxOd3dBTkZST1NWcEJUa0pCUVZCQlUwUlJOMEZGTkV0SVVGTk9VVVJSVGtsQ1FVdEVNa2RhTjB4SFdqY3lTMEUwUVVKTlZrOVJSMEU9fBbAdHgVXQvq4Dr8tuNUwjM4QgVAfW5LnQd3gz4M6ROQ"
NAMESPACE="kubeflow-user-example-com"
COOKIE="authservice_session="+AUTH
EXPERIMENT="Default"

## Set Log bucket and Tensorboard Image

In [7]:
MINIO_ENDPOINT="http://minio-service.kubeflow:9000"
LOG_BUCKET="mlpipeline"
TENSORBOARD_IMAGE="jagadeeshj/tb_plugin:v1.8"

In [8]:
client = kfp.Client(host=INGRESS_GATEWAY+"/pipeline", cookies=COOKIE)

In [9]:
client.create_experiment(EXPERIMENT)
experiments = client.list_experiments(namespace=NAMESPACE)
my_experiment = experiments.experiments[0]
my_experiment

{'created_at': datetime.datetime(2021, 4, 22, 8, 44, 39, tzinfo=tzlocal()),
 'description': None,
 'id': 'aac96a63-616e-4d88-9334-6ca8df2bb956',
 'name': 'Default',
 'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                  'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

## Set Inference parameters

In [10]:
DEPLOY_NAME="bertserve"
MODEL_NAME="bert"

In [11]:
prepare_tensorboard_op = load_component_from_file("./examples/common/tensorboard/component.yaml")
prep_op = components.load_component_from_file(
    "./examples/bert/yaml/pre_process/component.yaml"
)
train_op = components.load_component_from_file(
    "./examples/bert/yaml/train/component.yaml"
)
deploy_op = load_component_from_file("./examples/common/deploy/component.yaml")
minio_op = components.load_component_from_file(
    "./examples/common/minio/component.yaml"
)

## Define pipeline

In [12]:
@dsl.pipeline(name="Training pipeline", description="Sample training job test")
def pytorch_bert(
    minio_endpoint=MINIO_ENDPOINT,
    log_bucket=LOG_BUCKET,
    log_dir=f"tensorboard/logs/{dsl.RUN_ID_PLACEHOLDER}",
    mar_path=f"mar/{dsl.RUN_ID_PLACEHOLDER}/model-store",
    config_prop_path=f"mar/{dsl.RUN_ID_PLACEHOLDER}/config",
    model_uri=f"s3://mlpipeline/mar/{dsl.RUN_ID_PLACEHOLDER}",
    tf_image=TENSORBOARD_IMAGE,
    deploy=DEPLOY_NAME,
    namespace=NAMESPACE,
    confusion_matrix_log_dir=f"confusion_matrix/{dsl.RUN_ID_PLACEHOLDER}/",
    num_samples=1000
):

    prepare_tb_task = prepare_tensorboard_op(
        log_dir_uri=f"s3://{log_bucket}/{log_dir}",
        image=tf_image,
        pod_template_spec=json.dumps(
            {
                "spec": {
                    "containers": [
                        {
                            "env": [
                                {
                                    "name": "AWS_ACCESS_KEY_ID",
                                    "valueFrom": {
                                        "secretKeyRef": {
                                            "name": "mlpipeline-minio-artifact",
                                            "key": "accesskey",
                                        }
                                    },
                                },
                                {
                                    "name": "AWS_SECRET_ACCESS_KEY",
                                    "valueFrom": {
                                        "secretKeyRef": {
                                            "name": "mlpipeline-minio-artifact",
                                            "key": "secretkey",
                                        }
                                    },
                                },
                                {"name": "AWS_REGION", "value": "minio"},
                                {"name": "S3_ENDPOINT", "value": f"{minio_endpoint}"},
                                {"name": "S3_USE_HTTPS", "value": "0"},
                                {"name": "S3_VERIFY_SSL", "value": "0"},
                            ]
                        }
                    ]
                }
            }
        ),
    ).set_display_name("Visualization")

    prep_task = prep_op().after(prepare_tb_task).set_display_name("Preprocess & Transform")
    train_task = (
        train_op(
            input_data=prep_task.outputs["output_data"],
            profiler="pytorch",
            confusion_matrix_url=f"minio://{log_bucket}/{confusion_matrix_log_dir}",
            num_samples=num_samples
        )
        .after(prep_task)
        .set_display_name("Training")
    )

    minio_tb_upload = (
        minio_op(
            bucket_name="mlpipeline",
            folder_name=log_dir,
            input_path=train_task.outputs["tensorboard_root"],
            filename="",
        )
        .after(train_task)
        .set_display_name("Tensorboard Events Pusher")
    )
    minio_mar_upload = (
        minio_op(
            bucket_name="mlpipeline",
            folder_name=mar_path,
            input_path=train_task.outputs["checkpoint_dir"],
            filename="bert_test.mar",
        )
        .after(train_task)
        .set_display_name("Mar Pusher")
    )
    minio_config_upload = (
        minio_op(
            bucket_name="mlpipeline",
            folder_name=config_prop_path,
            input_path=train_task.outputs["checkpoint_dir"],
            filename="config.properties",
        )
        .after(train_task)
        .set_display_name("Conifg Pusher")
    )

    model_uri = str(model_uri)
    isvc_yaml = """
    apiVersion: "serving.kubeflow.org/v1beta1"
    kind: "InferenceService"
    metadata:
      name: {}
      namespace: {}
    spec:
      predictor:
        serviceAccountName: sa
        pytorch:
          storageUri: {}
          resources:
            limits:
              memory: 4Gi   
    """.format(
        deploy, namespace, model_uri
    )
    deploy_task = (
        deploy_op(action="apply", inferenceservice_yaml=isvc_yaml)
        .after(minio_mar_upload)
        .set_display_name("Deployer")
    )

    dsl.get_pipeline_conf().add_op_transformer(
        use_k8s_secret(
            secret_name="mlpipeline-minio-artifact",
            k8s_secret_key_to_env={
                "secretkey": "MINIO_SECRET_KEY",
                "accesskey": "MINIO_ACCESS_KEY",
            },
        )
    )


In [13]:
# Compile pipeline
compiler.Compiler().compile(pytorch_bert, 'pytorch.tar.gz', type_check=True)

/opt/conda/lib/python3.8/site-packages/kfp/components/_data_passing.py:169: UserWarning: Missing type name was inferred as "Integer" based on the value "1000".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))


In [14]:
# Execute pipeline
run = client.run_pipeline(my_experiment.id, 'pytorch-bert', 'pytorch.tar.gz')

## Wait for inference service below to go to `READY True` state.

In [15]:
!kubectl get isvc $DEPLOY

NAME         URL                                                       READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                  AGE
bertserve    http://bertserve.kubeflow-user-example-com.example.com    True           100                              bertserve-predictor-default-d9xbj    42h
torchserve   http://torchserve.kubeflow-user-example-com.example.com   True           100                              torchserve-predictor-default-wgqxj   7d16h


# Get Inferenceservice name

In [17]:
INFERENCE_SERVICE_LIST = ! kubectl get isvc $DEPLOY_NAME -n $NAMESPACE -o json | jq .status.url | tr -d '"'| cut -d "/" -f 3
INFERENCE_SERVICE_NAME = INFERENCE_SERVICE_LIST[0]
INFERENCE_SERVICE_NAME

'bertserve.kubeflow-user-example-com.example.com'

# Prediction Request

In [18]:
!curl -v -H "Host: $INFERENCE_SERVICE_NAME" -H "Cookie: $COOKIE" "$INGRESS_GATEWAY/v1/models/$MODEL_NAME:predict" -d @./examples/bert/sample.txt > bert_prediction_output.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/bert:predict HTTP/1.1
> Host: bertserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMjAzMTgwNHxOd3dBTkZST1NWcEJUa0pCUVZCQlUwUlJOMEZGTkV0SVVGTk9VVVJSVGtsQ1FVdEVNa2RhTjB4SFdqY3lTMEUwUVVKTlZrOVJSMEU9fBbAdHgVXQvq4Dr8tuNUwjM4QgVAfW5LnQd3gz4M6ROQ
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
} [84 bytes data]
* upload completely sent off: 84 out of 84 bytes
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 33
< content-type: application/json; charset=UTF-8
< date: Thu, 27 May 202

In [19]:
! cat bert_prediction_output.json

{"predictions": ["\"Sci/Tech\""]}

# Explanation Request

In [20]:
!curl -v -H "Host: $INFERENCE_SERVICE_NAME" -H "Cookie: $COOKIE" "$INGRESS_GATEWAY/v1/models/$MODEL_NAME:explain" -d @./examples/bert/sample.txt  > bert_explaination_output.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/bert:explain HTTP/1.1
> Host: bertserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMjAzMTgwNHxOd3dBTkZST1NWcEJUa0pCUVZCQlUwUlJOMEZGTkV0SVVGTk9VVVJSVGtsQ1FVdEVNa2RhTjB4SFdqY3lTMEUwUVVKTlZrOVJSMEU9fBbAdHgVXQvq4Dr8tuNUwjM4QgVAfW5LnQd3gz4M6ROQ
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
} [84 bytes data]
* upload completely sent off: 84 out of 84 bytes
100    84    0     0  100    84      0      1  0:01:24  0:00:53  0:00:31     0* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-leng

In [21]:
! cat bert_explaination_output.json

{"explanations": [{"words": ["[CLS]", "bloomberg", "has", "reported", "on", "the", "economy", "[SEP]"], "importances": [0.49801128883150986, -0.04228800481977043, -0.22690382385299707, 0.1557326112787248, 0.08676915006156335, 0.17918951470597463, 0.5255293995528972, -0.5988652369043819], "delta": 0.12079699740117719}]}

In [22]:
explanations_json = json.loads(open("./bert_explaination_output.json", "r").read())
explanations_json

{'explanations': [{'words': ['[CLS]',
    'bloomberg',
    'has',
    'reported',
    'on',
    'the',
    'economy',
    '[SEP]'],
   'importances': [0.49801128883150986,
    -0.04228800481977043,
    -0.22690382385299707,
    0.1557326112787248,
    0.08676915006156335,
    0.17918951470597463,
    0.5255293995528972,
    -0.5988652369043819],
   'delta': 0.12079699740117719}]}

In [23]:
prediction_json = json.loads(open("./bert_prediction_output.json", "r").read())

In [24]:
import torch
attributions = explanations_json["explanations"][0]['importances']
tokens = explanations_json["explanations"][0]['words']
delta = explanations_json["explanations"][0]['delta']

attributions = torch.tensor(attributions)
pred_prob = 0.75
pred_class = prediction_json["predictions"][0]
true_class = "Business"
attr_class ="world"

In [25]:
from captum.attr import visualization
vis_data_records =[]
vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred_prob,
                            pred_class,
                            true_class,
                            attr_class,
                            attributions.sum(),       
                            tokens,
                            delta))

In [26]:
visualization.visualize_text(vis_data_records)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Business,"""Sci/Tech"" (0.75)",world,0.58,[CLS] bloomberg has reported on the economy [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Business,"""Sci/Tech"" (0.75)",world,0.58,[CLS] bloomberg has reported on the economy [SEP]


## Cleanup Script

In [55]:
! kubectl delete --all viewers -n $NAMESPACE

In [56]:
! kubectl delete --all isvc -n $NAMESPACE

In [ ]:
! kubectl delete pod --field-selector=status.phase==Succeeded -n $NAMESPACE